# Base de datos neo4j

## Lectura de datos

In [1]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

## Conexión con neo4j y borrado de BB.DD.

In [2]:
%load_ext cypher

In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph()

# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en neo4j

 El modelo de nodos y relaciones que voy a considerar es este:
 
 ![Nodos y relaciones neo4j](./img/nodosRelacionesNEO.jpg)
 
 
 

Desnormalizo los datos de ascensos para facilitar la introducción de nodos y relaciones en la base de datos

In [4]:
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'inner');
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');

# Renombrar campos por claridad (el merge auto-asigna nombres a campos repetidos)
df_ascensos = df_ascensos.rename(
    columns={
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});

In [5]:
df_ascensos.head(2)

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais_risco,tipo_encadenamiento,grado_frances,grado_usa,grado_bloque_usa,nombre_escalador,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,anio_comienzo
0,20384,33,2,LE DE,FONTAINEBLEAU,ROCHER CANON,2017-04-17,FRA,Flash,5c,5.9,V2,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003
1,20384,33,2,SCHMALSEITE DV,ELBSANDSTEIN,KRALLENTURM,2017-06-10,DEU,Flash,5c,5.9,V2,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003


Convierto el data frame de ascensos desnormalizado a JSON para la inserción de información en neo4j

In [6]:
import json
json_string = df_ascensos.to_json(orient = 'records')
json_list = json.loads(json_string)

Itero en el data frame de ascensos añadiendo los nodos y las relaciones que se generan por cada ascenso:

In [7]:
for index, ascenso_json in enumerate(json_list):
    # Nodos
    escalador = Node("Escalador", nombre = ascenso_json["nombre_escalador"], anio_comienzo = ascenso_json["anio_comienzo"]) 
    via = Node("Via", nombre=ascenso_json["nombre_via"])
    dificultad = Node("Dificultad", valor = ascenso_json["id_dificultad"], grado_frances = ascenso_json["grado_frances"])
    risco = Node("Risco", nombre = ascenso_json["risco"])
    pais = Node("Pais", id=ascenso_json["pais_risco"])
    
  
        
    # Relaciones  
    rel_escalador_via = Relationship(escalador, "ASCIENDE", via, tipo_encadenamiento = ascenso_json["tipo_encadenamiento"])
    rel_escalador_pais = Relationship(escalador, "NACE", pais)
    rel_via_dificultad = Relationship(via, "POSEE", dificultad)
    rel_via_risco = Relationship(via, "PERTENECE", risco)   
    rel_risco_pais = Relationship(risco, "SE UBICA EN", pais)
    
    graph.merge(escalador|via|dificultad|risco|pais|rel_escalador_pais|rel_via_dificultad|rel_via_risco|rel_risco_pais)
    graph.create(rel_escalador_via)
    
    if index % 500 == 0:
        print index

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
